####  Aya AbdElmoniem Mohamed
####  Doaa Samir El-Sayed
####  Ahmed Mostafa AbdEl-Rahman
####  Mazen Gaber Mahmoud

# <center> Final KBAI Project </center>

# Table of Contents

- [Domain: Python Programming Education for Children](#domain-python-programming-education-for-children)
- [Installing Required Packages](#installing-required-packages)
- [Importing Required Modules](#importing-required-modules)
- [Phase 2: Collect Data with Web Scraping](#phase-2-collect-data-with-web-scraping)
- [Phase 3: Preprocess and Chunk the Text](#phase-3-preprocess-and-chunk-the-text)
- [Phase 4: Embed the Chunks](#phase-4-embed-the-chunks)
- [Phase 5: Create a Vector Store (ChromaDB)](#phase-5-create-a-vector-store-chromadb)
- [Phase 6: RAG System - Retrieval and Generation](#phase-6-rag-system---retrieval-and-generation)
- [Generating the Answers](#generating-the-answers)
- [Final Test: Sample Questions and Answers](#final-test-sample-questions-and-answers)


# Domain : Python Programming Education for Children
# Use Case :
This project creates a child-friendly Python programming assistant that explains coding concepts in simple, accessible ways for young learners aged 8-12. Programming education for children is increasingly important as digital literacy becomes essential, yet many resources use technical jargon that overwhelms young learners.

Our chatbot transforms complex programming concepts into age-appropriate explanations with relatable examples and encouraging language. It helps children understand fundamentals like variables, loops, and functions through analogies to everyday experiences. By making learning enjoyable and building confidence, this tool supports early coding education and helps develop computational thinking skills critical for future academic and career success.


## Installing Required Packages

In [ ]:
# 1. Install required packages if not already installed
!!pip install PyPDF2
!!pip install datasets
!pip install -q mistralai
!pip install -q chromadb sentence-transformers


## Importing Required Modules

In [ ]:
import os
import pickle
import numpy as np
import requests
import json
import time
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions
import pandas as pd
import random
import csv
import re
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
from tqdm import tqdm
import PyPDF2
import io


## Phase 2: Collect Data with Web Scraping

In [ ]:
# Create directory
os.makedirs("web_scraped_data__", exist_ok=True)

def clean_text(text):
    text = ' '.join(text.split())
    text = re.sub(r'\s+', ' ', text)
    return text

def extract_pdf_text(pdf_content):
    try:
        pdf_file = io.BytesIO(pdf_content)
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            page_text = page.extract_text() or ""
            text += page_text + "\n\n"
        return clean_text(text)
    except Exception as e:
        print(f"Error extracting PDF text: {str(e)}")
        return ""

def is_pdf_url(url):
    return url.lower().endswith('.pdf') or '/pdf/' in url.lower()

def scrape_url(url, min_paragraph_length=50):
    try:
        time.sleep(random.uniform(1, 3))
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
            'Accept': '*/*'
        }
        print(f"Scraping: {url}")
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()

        if is_pdf_url(url) or 'application/pdf' in response.headers.get('Content-Type', ''):
            print(f"Processing PDF: {url}")
            content = extract_pdf_text(response.content)
            title = os.path.basename(url) or "PDF Document"
            word_count = len(content.split())
            return {'url': url, 'title': title, 'content': content, 'word_count': word_count, 'status': 'success', 'type': 'pdf'}

        soup = BeautifulSoup(response.content, 'html.parser')
        title = soup.title.string if soup.title else "No Title"
        paragraphs = []
        for p in soup.find_all(['p', 'article', 'section', 'div.content', 'div.main']):
            text = p.get_text().strip()
            if len(text) >= min_paragraph_length:
                paragraphs.append(clean_text(text))
        content = '\n\n'.join(paragraphs)
        word_count = len(content.split())
        return {'url': url, 'title': title, 'content': content, 'word_count': word_count, 'status': 'success', 'type': 'html'}

    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return {'url': url, 'title': '', 'content': '', 'word_count': 0, 'status': f'error: {str(e)}', 'type': 'unknown'}

def find_pdf_links(url, html_content):
    pdf_links = []
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            if href.lower().endswith('.pdf'):
                if not href.startswith(('http://', 'https://')):
                    base_url = url.rstrip('/')
                    if href.startswith('/'):
                        href = f"{'/'.join(base_url.split('/')[:3])}{href}"
                    else:
                        href = f"{base_url}/{href}"
                pdf_links.append(href)
    except Exception as e:
        print(f"Error finding PDF links: {str(e)}")
    return pdf_links

# Seed URLs
seed_urls = [
    "https://www.programiz.com/python-programming/variables-constants-literals",
    "https://www.programiz.com/python-programming/numbers",
    "https://www.programiz.com/python-programming/if-elif-else",
    "https://www.programiz.com/python-programming/for-loop"
]

pdf_seed_urls = [
    "https://bugs.python.org/file47781/Tutorial_EDIT.pdf"
]

all_urls = seed_urls + pdf_seed_urls

all_scraped_data = []
total_word_count = 0
pdf_count = 0

for url in tqdm(all_urls, desc="Scraping URLs"):
    result = scrape_url(url)
    all_scraped_data.append(result)

    # Count PDF if successful
    if result['status'] == 'success' and result['type'] == 'pdf':
        pdf_count += 1

    if result['status'] == 'success' and result['type'] == 'html':
        try:
            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(url, headers=headers, timeout=10)
            pdf_links = find_pdf_links(url, response.content)
            for pdf_url in pdf_links:
                print(f"Found PDF link: {pdf_url}")
                pdf_result = scrape_url(pdf_url)
                all_scraped_data.append(pdf_result)
                if pdf_result['status'] == 'success' and pdf_result['type'] == 'pdf':
                    pdf_count += 1
        except Exception as e:
            print(f"Error processing PDF links from {url}: {str(e)}")

    if result['status'] == 'success':
        total_word_count += result['word_count']
        domain = urlparse(result['url']).netloc
        file_type = "pdf" if result['type'] == 'pdf' else "html"
        filename = f"{domain.replace('.', '_')}_{hash(result['url']) % 10000}.{file_type}.txt"
        filepath = os.path.join("web_scraped_data__", filename)

        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(f"URL: {result['url']}\n")
            f.write(f"Title: {result['title']}\n")
            f.write(f"Type: {result['type']}\n")
            f.write(f"Word Count: {result['word_count']}\n\n")
            f.write(result['content'])

csv_file = "web_scraped_data__/all_scraped_content.csv"
with open(csv_file, 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['url', 'title', 'content', 'word_count', 'status', 'type'])
    for item in all_scraped_data:
        truncated = item['content'][:1000] + '...' if len(item['content']) > 1000 else item['content']
        writer.writerow([item['url'], item['title'], truncated, item['word_count'], item['status'], item.get('type', 'unknown')])

with open("web_scraped_data__/all_content.txt", 'w', encoding='utf-8') as f:
    for item in all_scraped_data:
        if item['status'] == 'success':
            f.write(f"--- {item['title']} ({item.get('type', 'unknown')}) ---\n\n")
            f.write(item['content'])
            f.write("\n\n" + "="*80 + "\n\n")

successful_scrapes = sum(1 for item in all_scraped_data if item['status'] == 'success')
failed_scrapes = sum(1 for item in all_scraped_data if item['status'] != 'success')
html_count = sum(1 for item in all_scraped_data if item.get('type') == 'html' and item['status'] == 'success')

print(f"\nWeb Scraping Statistics:")
print(f"Total URLs scraped: {len(all_scraped_data)}")
print(f"Successful scrapes: {successful_scrapes}")
print(f"Failed scrapes: {failed_scrapes}")
print(f"HTML pages: {html_count}")
print(f"PDF documents: {pdf_count}")
print(f"Total words scraped: {total_word_count}")


Scraping URLs:   0%|          | 0/5 [00:00<?, ?it/s]

Scraping: https://www.programiz.com/python-programming/variables-constants-literals


Scraping URLs:  20%|██        | 1/5 [00:02<00:08,  2.09s/it]

Scraping: https://www.programiz.com/python-programming/numbers


Scraping URLs:  40%|████      | 2/5 [00:03<00:05,  1.98s/it]

...

Scraping: https://www.programiz.com/python-programming/for-loop


Scraping URLs:  80%|████████  | 4/5 [00:08<00:02,  2.14s/it]

Scraping: https://bugs.python.org/file47781/Tutorial_EDIT.pdf
Processing PDF: https://bugs.python.org/file47781/Tutorial_EDIT.pdf


Scraping URLs: 100%|██████████| 5/5 [00:14<00:00,  2.83s/it]


Web Scraping Statistics:
Total URLs scraped: 5
Successful scrapes: 5
Failed scrapes: 0
HTML pages: 4
PDF documents: 1
Total words scraped: 50726


## Phase 3: Preprocess and Chunk the Text

After collecting and cleaning data, we split it into smaller, manageable chunks. These chunks, typically 200–500 words, make it easier for embedding models to process text efficiently. This also improves the relevance and granularity of search results in later phases.


In [ ]:
def recursive_text_splitter(text, chunk_size=500, chunk_overlap=100):
    separators = ["\n\n", "\n", ". ", ", ", " ", ""]

    def _split(text, sep):
        if sep == "":
            return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size - chunk_overlap)]

        chunks, cur_chunk, cur_len = [], [], 0
        for part in text.split(sep):
            part = (sep + part) if cur_chunk else part
            if cur_len + len(part) > chunk_size and cur_chunk:
                joined = "".join(cur_chunk)
                chunks.append(joined)
                overlap = joined[-chunk_overlap:] if chunk_overlap < len(joined) else joined
                cur_chunk = [overlap, part]
                cur_len = len(overlap) + len(part)
            else:
                cur_chunk.append(part)
                cur_len += len(part)
        if cur_chunk:
            chunks.append("".join(cur_chunk))
        return chunks

    def _recursive(text, idx=0):
        if len(text) <= chunk_size or idx >= len(separators):
            return [text]
        split_chunks = _split(text, separators[idx])
        result = []
        for chunk in split_chunks:
            result.extend(_recursive(chunk, idx + 1) if len(chunk) > chunk_size else [chunk])
        return result

    return _recursive(text)

def split_documents(docs, chunk_size=500, chunk_overlap=100):
    return [
        {**doc, 'content': chunk, 'chunk_id': i}
        for doc in docs if doc.get('status') == 'success' and doc.get('content')
        for i, chunk in enumerate(recursive_text_splitter(doc['content'], chunk_size, chunk_overlap))
    ]


This function recursively splits a large block of text into smaller overlapping chunks, using progressively smaller separators (paragraphs, sentences, words, etc.). This ensures that each chunk stays within a desired length while maintaining natural language coherence.


In [ ]:
# After you've collected your web-scraped data
chunked_data = split_documents(all_scraped_data, chunk_size=500, chunk_overlap=100)

print(f"Original documents: {len(all_scraped_data)}")
print(f"After chunking: {len(chunked_data)} chunks")

Original documents: 5
After chunking: 1064 chunks


## Phase 4: Embed the Chunks

We use sentence embeddings to represent each chunk of text as a numerical vector. This enables us to measure semantic similarity between a question and text chunks.


In [ ]:
print("Computing embeddings for cleaned content...")
model = SentenceTransformer("all-MiniLM-L6-v2")

texts = [doc['content'] for doc in chunked_data]
embeddings = model.encode(texts, show_progress_bar=True)

print(f"Computed {len(embeddings)} embeddings")


Computing embeddings for cleaned content...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Batches:   0%|          | 0/34 [00:00<?, ?it/s]

Computed 1064 embeddings


This step initializes the MiniLM sentence transformer and encodes all chunked documents into vector representations. These vectors will later be used to find relevant chunks for answering user questions.

## Phase 5: Create a Vector Store (ChromaDB)

Chroma is a lightweight vector database used to store and retrieve text embeddings efficiently. This section builds a Chroma index using the previously computed embeddings and associated text chunks.


In [ ]:
# ====== Step 4: Initialize Chroma Client and Create Collection ======
client = chromadb.Client()


collection_name = f"python_for_kids_{int(time.time())}"
embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

collection = client.create_collection(
    name=collection_name,
    embedding_function=embedding_fn,
    metadata={"description": "Python programming content for children (in-memory)"}
)
print(f"Created new collection: {collection_name}")


Created new collection: python_for_kids_1746132699


In [ ]:
print("Adding documents to collection...")

ids = [f"chunk_{i}" for i in range(len(chunked_data))]
documents = texts
metadatas = [{
    'url': doc['url'],
    'title': doc['title'],
    'chunk_id': str(i),
    'word_count': str(doc['word_count']),
} for i, doc in enumerate(chunked_data)]

collection.add(
    ids=ids,
    documents=documents,
    metadatas=metadatas,
    embeddings=embeddings.tolist()
)
print(f"Added {len(documents)} documents to the collection.")


Adding documents to collection...
Added 1064 documents to the collection.


- `chromadb.Client()` initializes the database.
- A collection is created to group the chunks.
- Each chunk is added with metadata and a unique ID.
- The embedding function can be explicitly defined or computed externally.

In [ ]:
# Retrieve the stored embeddings directly
results = collection.peek()  # Returns first 10 items by default

for i in range(len(results["documents"])):
    print(f"\nDocument {i+1}: {results['documents'][i][:1000]}...")  # Truncated content
    print("Metadata:", results["metadatas"][i])
    print("Embedding:", results["embeddings"][i][:10], "...")       # Truncated vector




Document 1: Learn Python practically and Get Certified.

Created with over a decade of experience and thousands of feedback.

Learn Python practically and Get Certified.

In the previous tutorial you learned about Python comments. Now, let's learn about variables and literals in Python.

In programming, a variable is a container (storage area) to hold data. For example,

As we can see from the above example, we use the assignment operator = to assign a value to a variable....
Metadata: {'chunk_id': '0', 'title': 'Python Variables and Literals (With Examples)', 'url': 'https://www.programiz.com/python-programming/variables-constants-literals', 'word_count': '604'}
Embedding: [-0.00399923  0.00034872 -0.04260653  0.0943079  -0.08958005 -0.11675563
  0.06101482  0.05899188 -0.07405229  0.02892023] ...

Document 2: we can see from the above example, we use the assignment operator = to assign a value to a variable.

In the above example, we assigned the value programiz.pro to the site_name

## Phase 6: RAG System - Retrieval and Generation

This phase retrieves the most relevant text chunks from Chroma based on a user's question, then uses a language model like Mistral AI to generate an informative response using those chunks.


In [ ]:
def retrieve_relevant_chunks(query, n_results=5):
    """Retrieve the most relevant chunks for a query."""
    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )

    retrieved_chunks = []
    for doc, metadata, distance in zip(
        results['documents'][0],
        results['metadatas'][0],
        results['distances'][0]
    ):
        retrieved_chunks.append({
            'content': doc,
            'metadata': metadata,
            'relevance': 1-distance  # Higher means more relevant
        })

    return retrieved_chunks


In [ ]:
import inspect
with open("web_scraped_data__/retrieval_function.py", "w") as f:
    f.write(inspect.getsource(retrieve_relevant_chunks))
    f.write("\n\n# Example usage:\n")
    f.write("# from chromadb import PersistentClient\n")
    f.write("# client = PersistentClient(path='web_scraped_data/chroma_db')\n")
    f.write("# collection = client.get_collection(name='your_collection_name')\n")
    f.write("# results = retrieve_relevant_chunks('What is a variable?', collection)\n")

print("\nPhase 5 Complete: Vector store created with Chroma!")
print("we can now query your knowledge base using the retrieve_relevant_chunks() function.")


Phase 5 Complete: Vector store created with Chroma!
we can now query your knowledge base using the retrieve_relevant_chunks() function.


In [ ]:
retrieve_relevant_chunks("Variables in python?")

[{'content': "Learn Python practically and Get Certified.\n\nCreated with over a decade of experience and thousands of feedback.\n\nLearn Python practically and Get Certified.\n\nIn the previous tutorial you learned about Python comments. Now, let's learn about variables and literals in Python.\n\nIn programming, a variable is a container (storage area) to hold data. For example,\n\nAs we can see from the above example, we use the assignment operator = to assign a value to a variable.",
  'metadata': {'title': 'Python Variables and Literals (With Examples)',
   'word_count': '604',
   'chunk_id': '0',
   'url': 'https://www.programiz.com/python-programming/variables-constants-literals'},
  'relevance': 0.41486525535583496},
 {'content': "we can see from the above example, we use the assignment operator = to assign a value to a variable.\n\nIn the above example, we assigned the value programiz.pro to the site_name variable. Then, we printed out the value assigned to site_name\n\nNote: P

# Generating the Answers

In [ ]:
def format_context(chunks):
    """
    Format retrieved chunks into a string format to provide as context to the LLM.
    """
    context_parts = [
        f"Source {i+1} [{chunk['metadata']['title']}]:\n{chunk['content']}\n"
        for i, chunk in enumerate(chunks)
    ]
    return "\n".join(context_parts)


In [ ]:
def generate_answer_with_mistral(question, context, api_key=None):
    """
    Generate an answer using Mistral AI API, based on retrieved context.
    Falls back to a simple answer if the API fails.
    """
    if not api_key:
        api_key = os.getenv("MISTRAL_API_KEY")
        if not api_key:
            return "Error: Mistral API key not provided."

    url = "https://api.mistral.ai/v1/chat/completions"

    system_prompt = """You are a helpful, friendly AI assistant designed to teach Python programming to children.
Your answers should be:
1. Simple and easy to understand for children
2. Encouraging and positive
3. Accurate and based only on the provided context
4. Include simple examples when appropriate

If you don't know the answer based on the context, say so politely and suggest where they might find more information."""

    user_prompt = f"""Please answer the following question about Python programming for kids.
Use ONLY the information in the context provided below.

CONTEXT:
{context}

QUESTION: {question}"""

    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "mistral-small",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "temperature": 0.5,
        "max_tokens": 500
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        result = response.json()
        return result["choices"][0]["message"]["content"]

    except Exception as e:
        print(f"Error with Mistral API: {str(e)}")


In [ ]:
def rag_answer(question, api_key=None, n_chunks=3):
    """
    Complete RAG pipeline that retrieves context and generates an answer.
    """
    print(f"\nProcessing question: {question}")
    print("Retrieving relevant context...")
    chunks = retrieve_relevant_chunks(question, n_results=n_chunks)
    context = format_context(chunks)
    print(f"Retrieved {len(chunks)} relevant chunks")

    print("Generating answer with Mistral AI...")
    answer = generate_answer_with_mistral(question, context, api_key)

    return {
        'question': question,
        'answer': answer,
        'context_chunks': chunks
    }

In [ ]:
from IPython.display import display, Markdown
def displayAnswer(q):
  dicAnswer=rag_answer(q,"7CgDqJnql8ORf9bQqj1rvc4aDfEHo549")
  print("3 relevant chunks : ")
  structured_content = ""
  for item in dicAnswer['context_chunks']:
      structured_content += f"### {item['metadata']['title']}\n"
      structured_content += f"**Relevance**: {item['relevance']}\n"
      structured_content += f"**Word Count**: {item['metadata']['word_count']}\n"
      structured_content += f"[Read More]({item['metadata']['url']})\n"
      structured_content += f"\n{item['content']}\n\n---\n\n"

  display(Markdown(structured_content))
  print("Answer : ")
  display(Markdown(dicAnswer['answer']))

Overall, we:

1. Embeds the question using the same model as used for the document chunks.
2. Queries the Chroma vector store to retrieve the top relevant chunks.
3. Constructs a prompt combining the context chunks and the user question.
4. Passes the prompt to a generative language model (e.g., Mistral AI ) to generate an answer.
5. Displays:
   - The retrieved chunks with their relevance score and word count.
   - A link to the original source (if available).
   - The generated answer from the language model.

This allows users to verify the source context used to generate the response, improving transparency and trust.

# Final Test: Sample Questions and Answers

Below are five sample questions that were used to test the chatbot. For each, the system retrieves relevant context from the document store and generates an answer using the RAG pipeline.



In [ ]:
displayAnswer("what is While loop in python?")


Processing question: what is While loop in python?
Retrieving relevant context...
Retrieved 3 relevant chunks
Generating answer with Mistral AI...
3 relevant chunks : 


### Python for Loop (With Examples)
**Relevance**: 0.2761881947517395
**Word Count**: 541
[Read More](https://www.programiz.com/python-programming/for-loop)

e have created a list named languages. Since the list has three elements, the loop iterates 3 times.

The for loop iterates over the elements of sequence in order, and in each iteration, the body of the loop is executed.

The loop ends after the body of the loop is executed for the last item.

In Python, we use indentation (spaces at the beginning of a line) to define a block of code, such as the body of a loop. For example,

---

### Python for Loop (With Examples)
**Relevance**: 0.22722387313842773
**Word Count**: 541
[Read More](https://www.programiz.com/python-programming/for-loop)

aces at the beginning of a line) to define a block of code, such as the body of a loop. For example,

Here, print('Last statement') is outside the body of the loop. Therefore, this statement is executed only once at the end.

If we iterate through a string, we get individual characters of the string one by one.

Here, we have printed each character of the string language using a for loop.

In Python, the range() function returns a sequence of numbers. For example,

---

### Tutorial_EDIT.pdf
**Relevance**: 0.1915910243988037
**Word Count**: 48471
[Read More](https://bugs.python.org/file47781/Tutorial_EDIT.pdf)

t line contains a multiple assignment : the variables aandbsimultaneously get the new values 0 and 1. On the last line this is used again, demonstrating that the expressions on the right-hand side are all evaluated ﬁrst before any of the assignments take place. The right-hand side expressions are evaluated from the left to the right. •Thewhileloop executes as long as the condition (here: a < 10) remains true. In Python, like in C, any non-zero integer value is true; zero is false

---



Answer : 


A while loop in Python is a type of loop that keeps running a block of code as long as a certain condition is true. It's a bit different from the for loop we learned about earlier, which repeats a block of code a specific number of times.

With a while loop, you can tell Python to keep doing something until a specific condition is met. For example, you might use a while loop to ask a user for their name until they enter a valid response.

Here's an example of what a while loop might look like in Python:

while condition:
# this is the body of the loop

In this example, "condition" is a statement that evaluates to either True or False. As long as the condition is True, the body of the loop will keep running over and over again. Once the condition becomes False, the loop will stop.

Just like with for loops, it's important to use indentation to define the body of the while loop. Any code that you want to be repeated should be indented under the while statement.

Also, keep in mind that it's important to make sure that the condition will eventually become False, or else the loop will keep running forever! This is called an infinite loop, and it can cause problems if you're not careful.

I hope that helps! Let me know if you have any other questions about while loops in Python.

In [ ]:
displayAnswer("what is if conditions in Python?")


Processing question: what is if conditions in Python?
Retrieving relevant context...
Retrieved 3 relevant chunks
Generating answer with Mistral AI...
3 relevant chunks : 


### Python if, if...else Statement (With Examples)
**Relevance**: 0.45283710956573486
**Word Count**: 628
[Read More](https://www.programiz.com/python-programming/if-elif-else)

f user enters 10, the condition number > 0 evaluates to True. Therefore, the body of if is executed.

If user enters -2, the condition number > 0 evaluates to False. Therefore, the body of if is skipped from execution.

Python uses indentation to define a block of code, such as the body of an if statement. For example,

Here, the body of if has two statements. We know this because two statements (immediately after if) start with indentation.

---

### Python if, if...else Statement (With Examples)
**Relevance**: 0.44720160961151123
**Word Count**: 628
[Read More](https://www.programiz.com/python-programming/if-elif-else)

rtant to note that regardless the value of number variable, only one block of code will be executed.

It is possible to include an if statement inside another if statement. For example,

In certain situations, the if statement can be simplified into a single line. For example,

This one-liner approach retains the same functionality but in a more concise format.

Python doesn't have a ternary operator. However, we can use if...else to work like a ternary operator in other languages. For example,

---

### Python if, if...else Statement (With Examples)
**Relevance**: 0.37509429454803467
**Word Count**: 628
[Read More](https://www.programiz.com/python-programming/if-elif-else)

s two statements. We know this because two statements (immediately after if) start with indentation.

We usually use four spaces for indentation in Python, although any number of spaces works as long as we are consistent.

You will get an error if you write the above code like this:

Here, we haven't used indentation after the if statement. In this case, Python thinks our if statement is empty, which results in an error.

---



Answer : 


When we talk about "if conditions" in Python, we're referring to the concept of using the `if` keyword to make decisions based on whether a certain condition is true or false. For example, you might have an `if` statement that checks if a number is greater than 0. If that condition is true, then the code inside the `if` statement will run. If the condition is not true, then the code inside the `if` statement will be skipped over.

Here's a simple example:

```python
number = 10

if number > 0:
  print("The number is positive!")
```

In this case, the condition `number > 0` is true, so the message "The number is positive!" will be printed to the screen.

It's also worth noting that Python uses indentation to define blocks of code. In the example above, the `print` statement is indented to show that it's part of the `if` statement's block of code. If we didn't include that indentation, we would get an error.

I hope that helps! Let me know if you have any other questions about `if` conditions in Python.

In [ ]:
displayAnswer("what is variable meaning in python?")


Processing question: what is variable meaning in python?
Retrieving relevant context...
Retrieved 3 relevant chunks
Generating answer with Mistral AI...
3 relevant chunks : 


### Python Variables and Literals (With Examples)
**Relevance**: 0.4003611207008362
**Word Count**: 604
[Read More](https://www.programiz.com/python-programming/variables-constants-literals)

Learn Python practically and Get Certified.

Created with over a decade of experience and thousands of feedback.

Learn Python practically and Get Certified.

In the previous tutorial you learned about Python comments. Now, let's learn about variables and literals in Python.

In programming, a variable is a container (storage area) to hold data. For example,

As we can see from the above example, we use the assignment operator = to assign a value to a variable.

---

### Tutorial_EDIT.pdf
**Relevance**: 0.28519517183303833
**Word Count**: 48471
[Read More](https://bugs.python.org/file47781/Tutorial_EDIT.pdf)

ntaining Python deﬁnitions and statements. The ﬁle name is the module name with the suﬃx .pyappended. Within a module, the module’s name (as a string) is available as the value of the global variable __name__ 

---

### Python Variables and Literals (With Examples)
**Relevance**: 0.2008957862854004
**Word Count**: 604
[Read More](https://www.programiz.com/python-programming/variables-constants-literals)

we can see from the above example, we use the assignment operator = to assign a value to a variable.

In the above example, we assigned the value programiz.pro to the site_name variable. Then, we printed out the value assigned to site_name

Note: Python is a type-inferred language, so you don't have to explicitly define the variable type. It automatically knows that programiz.pro is a string and declares the site_name variable as a string.

---



Answer : 


Hello! I'm here to help you with Python programming questions.

A variable in Python is like a container where you can store data. You can think of it as a label for a value that you want to use in your program. For example, you might have a variable called 'age' that stores your age as a number.

In Python, you don't have to tell the computer what type of data a variable will store. The computer can figure it out on its own. So if you assign the string 'programiz.pro' to a variable called 'site_name', Python will automatically know that 'site_name' is a string.

Here's an example:

site\_name = 'programiz.pro'
print(site\_name)

In this example, we use the assignment operator '=' to assign the value 'programiz.pro' to the variable 'site\_name'. Then we use the 'print' function to display the value of 'site\_name'.

I hope that helps! Let me know if you have any other questions.

In [ ]:
displayAnswer("what is Types of Numbers in Python?")


Processing question: what is Types of Numbers in Python?
Retrieving relevant context...
Retrieved 3 relevant chunks
Generating answer with Mistral AI...
3 relevant chunks : 


### Python Numbers, Type Conversion and Mathematics (With Examples)
**Relevance**: 0.5917428433895111
**Word Count**: 482
[Read More](https://www.programiz.com/python-programming/numbers)

Learn Python practically and Get Certified.

Created with over a decade of experience and thousands of feedback.

Learn Python practically and Get Certified.

The number data types are used to store the numeric values.

Python supports integers, floating-point numbers and complex numbers. They are defined as int, float, and complex classes in Python.

Integers and floating points are separated by the presence or absence of a decimal point. For instance,

---

### Tutorial_EDIT.pdf
**Relevance**: 0.46278780698776245
**Word Count**: 48471
[Read More](https://bugs.python.org/file47781/Tutorial_EDIT.pdf)

 numbers (e.g. 2,4,20) have type int, the ones with a fractional part (e.g. 5.0,1.6) have type float. We will see more about numeric types later in the tutorial. 9 Python Tutorial, Release 3.7.0 Division ( /) always returns a ﬂoat

---

### Python Numbers, Type Conversion and Mathematics (With Examples)
**Relevance**: 0.3217265009880066
**Word Count**: 482
[Read More](https://www.programiz.com/python-programming/numbers)

grammers need to work with binary (base 2), hexadecimal (base 16) and octal (base 8) number systems.

In Python, we can represent these numbers by appropriately placing a prefix before that number. The following table lists these prefixes.

In programming, type conversion is the process of converting one type of number into another.

Operations like addition, subtraction convert integers to float implicitly (automatically), if one of the operands is float. For example,

---



Answer : 


In Python, there are different types of numbers that you can work with! There are integers (like 5, 10, 20), which are numbers without any decimal points. Floating-point numbers (like 5.0, 1.6) have a decimal point, and you might also come across complex numbers (like 2+3j) which involve imaginary parts.

Sometimes, you might need to work with other number systems like binary (base 2), hexadecimal (base 16), or octal (base 8). In Python, you can represent these numbers by placing a prefix before them. Here's a table to help you remember:

- Binary: 0b
- Hexadecimal: 0x
- Octal: 0o

As you learn more about Python, you'll find that there are operations where converting one type of number to another happens automatically. For example, when you do addition or subtraction, and if one of the operands is a float, then Python will convert the integer to a float automatically.

Keep practicing and having fun learning Python! You're doing a great job!

In [ ]:
displayAnswer("how to loop in Python?")


Processing question: how to loop in Python?
Retrieving relevant context...
Retrieved 3 relevant chunks
Generating answer with Mistral AI...
3 relevant chunks : 


### Python for Loop (With Examples)
**Relevance**: 0.266865611076355
**Word Count**: 541
[Read More](https://www.programiz.com/python-programming/for-loop)

e have created a list named languages. Since the list has three elements, the loop iterates 3 times.

The for loop iterates over the elements of sequence in order, and in each iteration, the body of the loop is executed.

The loop ends after the body of the loop is executed for the last item.

In Python, we use indentation (spaces at the beginning of a line) to define a block of code, such as the body of a loop. For example,

---

### Python for Loop (With Examples)
**Relevance**: 0.16251826286315918
**Word Count**: 541
[Read More](https://www.programiz.com/python-programming/for-loop)

Learn Python practically and Get Certified.

Created with over a decade of experience and thousands of feedback.

Learn Python practically and Get Certified.

In Python, we use a for loop to iterate over sequences such as lists, strings, dictionaries, etc. For example,

In the above example, we have created a list named languages. Since the list has three elements, the loop iterates 3 times.

---

### Tutorial_EDIT.pdf
**Relevance**: 0.13413816690444946
**Word Count**: 48471
[Read More](https://bugs.python.org/file47781/Tutorial_EDIT.pdf)

dioms of the Python language, rather than implementing code using concepts common to other languages. For example, a common idiom in Python is to loop over all elements of an iterable using a forstatement. Many other languages don’t have this type of construct, so people unfamiliar with Python sometimes use a numerical counter instead: foriinrange(len(food)): print(food[i]) 122 Appendix A

---



Answer : 


To loop in Python, you can use a "for" loop! This kind of loop is used to iterate over sequences such as lists, strings, or dictionaries. Here's a simple example of how to use a "for" loop with a list:

```python
languages = ["Python", "Java", "C++"]
for language in languages:
    print(language)
```

In this example, the loop goes through each element in the "languages" list one-by-one. For every element, it executes the indented code below it, which is printing the current language. This way, you can easily loop through and do something with each item in a sequence!

By the way, a helpful tip from the context is that in Python, we use indentation (spaces at the beginning of a line) to define a block of code, like the body of a loop. So, remember to use indentation correctly when writing your "for" loops!